<a href="https://colab.research.google.com/github/Sa-Fi/Crop-pest-and-disease-detection/blob/main/Crop_diseases_using_machine_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#upload the kaggle json
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!kaggle datasets download -d safiullah007/leaf-disease



100% 2.01G/2.01G [01:23<00:00, 27.7MB/s]
100% 2.01G/2.01G [01:23<00:00, 25.8MB/s]


In [ ]:
# unzip the file
import zipfile
zip_ref = zipfile.ZipFile('/content/leaf-disease.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [2]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D,Flatten,Dense,MaxPooling2D,Dropout
from sklearn.metrics import accuracy_score
import ipywidgets as widgets
import io
from PIL import Image
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import cv2
from sklearn.utils import shuffle
import tensorflow as tf
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Define classes and subclasses
classes = ['Cashew', 'Cassava', 'Maize', 'Tomato']
subclasses = {
    'Cashew': ['anthracnose', 'gumosis', 'healthy', 'leaf miner', 'red rust'],
    'Cassava': ['bacterial blight', 'brown spot', 'green mite', 'healthy', 'mosaic'],
    'Maize': ['fall armyworm', 'grasshoper', 'healthy', 'leaf beetle', 'leaf blight', 'leaf spot', 'streak virus'],
    'Tomato': ['healthy', 'leaf blight', 'leaf curl', 'septoria leaf spot', 'verticulium wilt']
}

data = []  # To store images as arrays
labels = []  # To store labels
progress_bar = tqdm(total=len(classes) * len(subclasses))
main_folder = '/content/CCMT Dataset/CCMT Dataset'

# Read images and assign labels
for class_name in classes:
    for subclass_name in subclasses[class_name]:
        folder_path = os.path.join(main_folder, class_name, subclass_name)
        for filename in os.listdir(folder_path):
           # if filename.endswith('.jpg') or filename.endswith('.png'):
                image_path = os.path.join(folder_path, filename)
                # image = Image.open(image_path)
                image = cv2.imread(image_path)
                if image is not None:
                  image = cv2.resize(image,(150,150))
                  image_array = np.array(image)
                  # image_array = normalize_image(image)
                  data.append(image_array)
                  labels.append(class_name + '_' + subclass_name)
                  progress_bar.update(1)
                else:
                  print(f"Skipping file: {image_path}")
progress_bar.close()

# Convert 'data' and 'labels' lists to NumPy arrays
data = np.array(data)
labels = np.array(labels)


15075it [00:52, 321.82it/s]

Skipping file: /content/CCMT Dataset/CCMT Dataset/Maize/healthy/healthy88_.jpg


15258it [00:52, 330.61it/s]

Skipping file: /content/CCMT Dataset/CCMT Dataset/Maize/healthy/healthy18_.jpg


15559it [00:53, 336.33it/s]

...

21331it [01:13, 300.13it/s]

Skipping file: /content/CCMT Dataset/CCMT Dataset/Tomato/leaf curl/leaf curl43_.jpg
Skipping file: /content/CCMT Dataset/CCMT Dataset/Tomato/leaf curl/leaf curl186_.jpg


21677it [01:14, 334.37it/s]

Skipping file: /content/CCMT Dataset/CCMT Dataset/Tomato/leaf curl/leaf curl271_.jpg


25170it [01:26, 292.23it/s]


In [ ]:
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

In [ ]:
x = (data)
y = (labels)
print("type of x independent varible which contain images :",type(x))
print("type of y dependent variable which contain the image label (with mask/without mask)",type(y))

type of x independent varible which contain images : <class 'numpy.ndarray'>
type of y dependent variable which contain the image label (with mask/without mask) <class 'numpy.ndarray'>


In [3]:
loaded_data = np.load('/content/drive/MyDrive/data store/data.npy')
loaded_labels = np.load('/content/drive/MyDrive/data store/labels.npy')

In [4]:
x = (loaded_data)
y = (loaded_labels)
print("type of x independent varible which contain images :",type(x))
print("type of y dependent variable which contain the image label (with mask/without mask)",type(y))

type of x independent varible which contain images : <class 'numpy.ndarray'>
type of y dependent variable which contain the image label (with mask/without mask) <class 'numpy.ndarray'>


In [5]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.1,random_state=2)

In [6]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()
y_train_encoded = encoder.fit_transform(y_train.reshape(-1, 1)).toarray()
y_test_encoded = encoder.fit_transform(y_test.reshape(-1, 1)).toarray()

In [37]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
def performace_measure(y_pred,y_test):
  conf_matrix = confusion_matrix(y_test, y_pred)

  # Calculate accuracy
  accuracy = accuracy_score(y_test, y_pred)

  # Calculate precision
  precision = precision_score(y_test, y_pred,average='micro')

  # Calculate recall
  recall = recall_score(y_test, y_pred,average='micro')

  # Calculate F1 score
  f1 = f1_score(y_test, y_pred,average='micro')

  # Print the results
  # print("Confusion Matrix:")
  # print(conf_matrix)
  print("\nAccuracy:", accuracy)
  print("Precision:", precision)
  print("Recall:", recall)
  print("F1 Score:", f1)


In [33]:
# machine learning algorithm function
from sklearn.metrics import accuracy_score,confusion_matrix

from sklearn.naive_bayes import GaussianNB
def GaussianNB_Custom(x_train,x_test,y_train,y_test):
  gnb = GaussianNB()
  gnb.fit(x_train,y_train)
  y_pred = gnb.predict(x_test)
  print("\n\n")
  print('performance measure : GaussianNB')
  performace_measure(y_test,y_pred)
  print("Accuracy for Gausssian NB",accuracy_score(y_test,y_pred))


# random forest Classifier
from sklearn.ensemble import RandomForestClassifier
def Random_Forest_Classifier_Custom(x_train,x_test,y_train,y_test):
  rf=RandomForestClassifier()
  rf.fit(x_train,y_train)
  y_pred = rf.predict(x_test)
  print("\n\n")
  print('performance measure : RandomForestClassifier')
  performace_measure(y_test,y_pred)
  print("Accuracy for Random forest ",accuracy_score(y_test,y_pred))

# K-Nearest Neighbors Clasifier Algorithm
from sklearn.neighbors import KNeighborsClassifier
def K_Nearest_Neighbors_Clasifier_Custom(x_train,x_test,y_train,y_test):
  KNC = KNeighborsClassifier(n_neighbors=77)
  KNC.fit(x_train,y_train)
  y_pred = KNC.predict(x_test)
  print("\n\n")
  print('performance measure : KNeighborsClassifier')
  performace_measure(y_test,y_pred)
  print("Accuracy for K-Nearest Neighbors Clasifier ",accuracy_score(y_test,y_pred))

# Support vector machine Classifier
from sklearn.svm import SVC
def Support_Vector_Machine_Classifier(x_train,x_test,y_train,y_test):
  svmc = SVC()
  svmc.fit(x_train,y_train)
  y_pred = svmc.predict(x_test)
  print("\n\n")
  print('performance measure : Support_Vector_Machine')
  performace_measure(y_test,y_pred)
  print("Accuracy for Support vector machine Classifier  ",accuracy_score(y_test,y_pred))

# BernoulliNB
from sklearn.naive_bayes import BernoulliNB
def BernoulliNB_Custom(x_train,x_test,y_train,y_test):
  BNB = BernoulliNB()
  BNB.fit(x_train,y_train)
  y_pred = BNB.predict(x_test)
  print("\n\n")
  print('performance measure : BernoulliNB')
  performace_measure(y_test,y_pred)
  print("Accuracy for BernoulliNB ",accuracy_score(y_test,y_pred))

# MultinomialNB
from sklearn.naive_bayes import  MultinomialNB
def MultinomialNB_Custom(x_train,x_test,y_train,y_test):
  MNB = MultinomialNB()
  MNB.fit(x_train,y_train)
  y_pred = MNB.predict(x_test)
  print("\n\n")
  print('performance measure : MultinomialNB')
  performace_measure(y_test,y_pred)
  print("Accuracy for MultinomialNB ",accuracy_score(y_test,y_pred))

# LogisticRegression
from sklearn.linear_model import LogisticRegression
def LogisticRegression_Custom(x_train,x_test,y_train,y_test):
  LR = LogisticRegression()
  LR.fit(x_train,y_train)
  y_pred = LR.predict(x_test)
  print("\n\n")
  print('performance measure : LogisticRegression')
  performace_measure(y_test,y_pred)
  print("Accuracy for LogisticRegression  ",accuracy_score(y_test,y_pred))


In [ ]:
import numpy as np

# Save data and labels to disk
np.save('/content/drive/MyDrive/data store/data.npy', data)
np.save('/content/drive/MyDrive/data store/labels.npy', labels)

In [ ]:
loaded_data = np.load('/content/drive/MyDrive/data store/data.npy')
loaded_labels = np.load('/content/drive/MyDrive/data store/labels.npy')

In [ ]:
type(loaded_data[0])

numpy.ndarray

In [ ]:
import cv2

# Resize images to a smaller size
resized_x_train = [cv2.resize(image, (64, 64)) for image in x_train]
resized_x_test = [cv2.resize(image, (64, 64)) for image in x_test]

# Flatten the resized images
x_train_flat = np.array([image.flatten() for image in resized_x_train])
x_test_flat = np.array([image.flatten() for image in resized_x_test])

# Continue with your Random Forest Classifier code



In [ ]:

Random_Forest_Classifier_Custom(x_train_flat, x_test_flat, y_train, y_test)

K_Nearest_Neighbors_Clasifier_Custom(x_train_flat, x_test_flat, y_train, y_test)

Support_Vector_Machine_Classifier(x_train_flat,x_test_flat,y_train,y_test)
BernoulliNB_Custom(x_train_flat,x_test_flat,y_train,y_test)

MultinomialNB_Custom(x_train_flat,x_test_flat,y_train,y_test)


GaussianNB_Custom(x_train_flat,x_test_flat,y_train,y_test)


Accuracy for Random forest  0.4692093762415574
Accuracy for K-Nearest Neighbors Clasifier  0.3170441001191895
Accuracy for Support vector machine Classifier   0.4858959078267779
Accuracy for BernoulliNB  0.11164084227254668
Accuracy for MultinomialNB  0.19189511323003575
Accuracy for Gausssian NB 0.20659515295987285


In [9]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Model

# Load the VGG16 model without the top (fully connected) layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Create a new model using the VGG16 layers as the feature extractor
feature_extractor = Model(inputs=base_model.input, outputs=base_model.layers[-1].output)
import numpy as np

def extract_features(data):
    # Preprocess the data
    # data = preprocess_input(data)

    # Extract features using the feature extractor
    features = feature_extractor.predict(data)

    return features
from sklearn.model_selection import train_test_split

# Split your dataset into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Extract features from the training and testing data
# x_train_features = extract_features(x_train)
# x_test_features = extract_features(x_test)


In [10]:
x_train_features = extract_features(x_train)
x_test_features = extract_features(x_test)
x_train_vgg16 = np.array([image.flatten() for image in x_train_features])
x_test_vgg16 = np.array([image.flatten() for image in x_test_features])

158/158 [==============================] - 11s 67ms/step


In [14]:

Random_Forest_Classifier_Custom(x_train_vgg16, x_test_vgg16, y_train, y_test)

K_Nearest_Neighbors_Clasifier_Custom(x_train_vgg16, x_test_vgg16, y_train, y_test)

Support_Vector_Machine_Classifier(x_train_vgg16,x_test_vgg16,y_train,y_test)
BernoulliNB_Custom(x_train_vgg16,x_test_flat,y_train,y_test)

MultinomialNB_Custom(x_train_vgg16,x_test_vgg16,y_train,y_test)


GaussianNB_Custom(x_train_vgg16,x_test_vgg16,y_train,y_test)


Accuracy for Random forest  0.6750099324592769
Accuracy for K-Nearest Neighbors Clasifier  0.45152959872864523
Accuracy for Support vector machine Classifier   0.7850615812475169
Accuracy for BernoulliNB  0.5834326579261025
Accuracy for MultinomialNB  0.624751688518077
Accuracy for Gausssian NB 0.46146205800556217


In [19]:
model = Sequential()

# Add Convolutional layers
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPooling2D(2, 2))
model.add(Conv2D(96, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(Conv2D(96, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(Conv2D(96, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))

# Flatten the output for feeding into fully connected layers
model.add(Flatten())

# Add fully connected layers
model.add(Dense(96, activation='relu'))
model.add(Dense(22, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Assuming you have a dataset 'x_train' and 'y_train'
# Train the model
model.fit(x_train, y_train_encoded, epochs=5, batch_size=32, validation_split = 0.1)

# Extract features using the modified CNN architecture
def extract_features(data):
    features = model.predict(data)
    return features


Epoch 1/5
567/567 [==============================] - 21s 26ms/step - loss: 3.1217 - accuracy: 0.1039 - val_loss: 2.9349 - val_accuracy: 0.1097
Epoch 2/5
567/567 [==============================] - 14s 25ms/step - loss: 2.9475 - accuracy: 0.1056 - val_loss: 2.9357 - val_accuracy: 0.1072
Epoch 3/5
567/567 [==============================] - 14s 25ms/step - loss: 2.9448 - accuracy: 0.1050 - val_loss: 2.9339 - val_accuracy: 0.1172
Epoch 4/5
567/567 [==============================] - 14s 25ms/step - loss: 2.9422 - accuracy: 0.1055 - val_loss: 2.9318 - val_accuracy: 0.1187
Epoch 5/5
567/567 [==============================] - 14s 25ms/step - loss: 2.9404 - accuracy: 0.1064 - val_loss: 2.9331 - val_accuracy: 0.1187


In [27]:
model = Sequential()

model.add(Conv2D(32,(3,3),activation = 'relu',input_shape=(150,150,3)))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(64,(3,3),activation = 'relu'))
model.add(MaxPooling2D(2,2))
layers.Conv2D(64, (3, 3), activation='relu')
model.add(MaxPooling2D(2,2))
layers.Conv2D(64, (3, 3), activation='relu')
model.add(MaxPooling2D(2,2))
layers.Conv2D(64, (3, 3), activation='relu')
model.add(MaxPooling2D(2,2))

model.add(Flatten())

model.add(Dense(64,activation = 'relu'))
model.add(Dense(22,activation='softmax'))
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(x_train,
                    y_train,
                    validation_split = 0.1,
                    epochs = 10)

Epoch 1/10
567/567 [==============================] - 10s 16ms/step - loss: 3.6381 - accuracy: 0.1068 - val_loss: 2.9955 - val_accuracy: 0.1216
Epoch 2/10
567/567 [==============================] - 9s 15ms/step - loss: 2.9781 - accuracy: 0.1086 - val_loss: 2.9510 - val_accuracy: 0.1216
Epoch 3/10
567/567 [==============================] - 9s 15ms/step - loss: 2.9524 - accuracy: 0.1087 - val_loss: 2.9345 - val_accuracy: 0.1216
Epoch 4/10
567/567 [==============================] - 9s 15ms/step - loss: 2.9434 - accuracy: 0.1087 - val_loss: 2.9285 - val_accuracy: 0.1216
Epoch 5/10
567/567 [==============================] - 9s 15ms/step - loss: 2.9401 - accuracy: 0.1087 - val_loss: 2.9263 - val_accuracy: 0.1216
Epoch 6/10
567/567 [==============================] - 9s 15ms/step - loss: 2.9388 - accuracy: 0.1087 - val_loss: 2.9258 - val_accuracy: 0.1216
Epoch 7/10
567/567 [==============================] - 9s 15ms/step - loss: 2.9382 - accuracy: 0.1087 - val_loss: 2.9253 - val_accuracy: 0.121

In [28]:
def extract_features(data):
    features = model.predict(data)
    return features
x_train_features = extract_features(x_train)
x_test_features = extract_features(x_test)
x_train_cnn = np.array([image.flatten() for image in x_train_features])
x_test_cnn = np.array([image.flatten() for image in x_test_features])

158/158 [==============================] - 1s 5ms/step


In [38]:

Random_Forest_Classifier_Custom(x_train_cnn, x_test_cnn, y_train, y_test)

K_Nearest_Neighbors_Clasifier_Custom(x_train_cnn, x_test_cnn, y_train, y_test)

Support_Vector_Machine_Classifier(x_train_cnn,x_test_cnn,y_train,y_test)
BernoulliNB_Custom(x_train_cnn,x_test_cnn,y_train,y_test)

MultinomialNB_Custom(x_train_cnn,x_test_cnn,y_train,y_test)


GaussianNB_Custom(x_train_cnn,x_test_cnn,y_train,y_test)




performance measure : RandomForestClassifier
Confusion Matrix:
[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0]
 [  

In [39]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.models import Model
import numpy as np

# Load the InceptionV3 model without the top (fully connected) layers
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Create a new model using the InceptionV3 layers as the feature extractor
feature_extractor = Model(inputs=base_model.input, outputs=base_model.layers[-1].output)

def extract_features(data):
    # Preprocess the data
    data = preprocess_input(data)

    # Extract features using the feature extractor
    features = feature_extractor.predict(data)

    return features


87910968/87910968 [==============================] - 0s 0us/step


In [40]:
def extract_features(data):
    features = model.predict(data)
    return features
x_train_features = extract_features(x_train)
x_test_features = extract_features(x_test)
x_train_inception = np.array([image.flatten() for image in x_train_features])
x_test_inception = np.array([image.flatten() for image in x_test_features])

158/158 [==============================] - 1s 4ms/step


In [41]:

Random_Forest_Classifier_Custom(x_train_inception, x_test_inception, y_train, y_test)

K_Nearest_Neighbors_Clasifier_Custom(x_train_inception, x_test_inception, y_train, y_test)

Support_Vector_Machine_Classifier(x_train_inception,x_test_inception,y_train,y_test)
BernoulliNB_Custom(x_train_inception,x_test_inception,y_train,y_test)

MultinomialNB_Custom(x_train_inception,x_test_inception,y_train,y_test)


GaussianNB_Custom(x_train_inception,x_test_inception,y_train,y_test)





performance measure : RandomForestClassifier
Confusion Matrix:
[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0]
 [  